import libraries

In [1]:
from besos import eppy_funcs as ef
from besos import sampling
from besos.evaluator import EvaluatorEP
from besos.parameters import RangeParameter, wwr, FieldSelector, Parameter
from besos.objectives import MeterReader, VariableReader, clear_outputs
from besos.problem import EPProblem, Problem
from besos.optimizer import NSGAII
import pandas as pd
import matplotlib.pyplot as plt

# Objectives and Constraints

`Evaluators` support two types of outputs: Objectives and Constraints.  
These are both made using the `MeterReader` and `VariableReader` classes.  
The only difference is how they are used by the problem.

First we load the EnergyPlus example file, clear any output data and define some parameters.

In [2]:
building = ef.get_building()
clear_outputs(building)
inputs = [wwr(), Parameter(FieldSelector(class_name='Material',
                                         object_name='Mass NonRes Wall Insulation',
                                         field_name='Thickness'),
                           RangeParameter(0.01, 0.99))
         ]

Objectives and constraints can be specified in various ways.
+ The most explicit is by calling the relevant constructor.

In [3]:
objectives = [MeterReader(key_name='Electricity:Facility', class_name='Output:Meter', frequency='Hourly')]
EPProblem(outputs=objectives)

EPProblem(outputs=[MeterReader(class_name='Output:Meter', frequency='Hourly', func=<function sum_values at 0x7f06a0ffcf28>, key_name='Electricity:Facility')], minimize_outputs=[True], converters={'outputs': <class 'objectives.MeterReader'>, 'constraints': <class 'objectives.MeterReader'>})

+ The most concise is a list of the `key_names`.

The constructor has defaults, so we can often omit `class_name` and `frequency`.  
A list of key names will be automatically be converted by `EPProblem`.  
Meters and variables that do not have a `frequency` specified will default to any frequency that is already used for that output, or if none is used yet then they will use Hourly.

In [4]:
objectives = ['Electricity:Facility']
EPProblem(outputs=objectives)

EPProblem(outputs=[MeterReader(class_name='Output:Meter', func=<function sum_values at 0x7f06b435ef28>, key_name='Electricity:Facility')], minimize_outputs=[True], converters={'outputs': <class 'objectives.MeterReader'>, 'constraints': <class 'objectives.MeterReader'>})

+ Using `Problem`

If we do not need the output-reading features of meters, we can use `Problem` instead of `EPProblem`, and they will be converted to `Objective` objects which act as placeholders.  
`EPProblem` converts them to `Meter:Reader` objects.  
Either of these conversions can be overriden using the converters argument.  

In [5]:
objectives = ['any', 'names', 'work']
Problem(outputs=objectives)

Problem(outputs=[Objective(name='any'), Objective(name='names'), Objective(name='work')], minimize_outputs=[True, True, True], converters={'outputs': <class 'IO_Objects.Objective'>, 'constraints': <class 'IO_Objects.Objective'>})

+ Specifying the aggregation function

The `func` argument is used define how to aggregate the individual time series results.  
By default, all measurements are summed.  
If we wanted to instead minimize the variance, we can write our own aggrgation function.  
Here we define two electricity objectives, the first summing the hourly values and the second taking the variance.

In [6]:
def variance(result):
    return result.data['Value'].var()

objectives = [MeterReader('Electricity:Facility', name='Electricity Usage'),
              MeterReader('Electricity:Facility',func=variance, name='Electricity Variance')
             ]

When we want to specify the direction of optimisation, we can use `minmize_outputs` (defaults to `true` for all objectives).  
Here we say we want to search for a design that has:
+ low electricty use (minimize objective 1 defined above)
+ high variability of electricity use (maximize objective 2 defined above)
+ less than 800 kgCO2 (constraint)

In [7]:
evaluator = EvaluatorEP(EPProblem(inputs=inputs,
                                  outputs=objectives, minimize_outputs=[True, True],
                                  constraints=['CO2:Facility'], constraint_bounds=['<=800']),
                                  building, out_dir='outputdir')

In [8]:
# this cell runs the optimisation
results1 = NSGAII(evaluator, evaluations=1, population_size=10)
results1

,Window to Wall Ratio,Thickness,Electricity Usage,Electricity Variance,CO2:Facility,violation,pareto-optimal
0,0.258835,0.329794,1.759326e+09,7.059078e+14,695.213518,0,False
1,0.775190,0.498031,1.753973e+09,7.038401e+14,690.079221,0,False
2,0.630642,0.591687,1.751086e+09,7.022339e+14,688.330490,0,False
3,0.321375,0.607029,1.751668e+09,7.028930e+14,688.429672,0,False
4,0.800154,0.924937,1.745048e+09,7.002741e+14,685.014693,0,True
5,0.163903,0.229399,1.765658e+09,7.091754e+14,698.700024,0,False
6,0.389494,0.461003,1.754765e+09,7.039509e+14,690.937572,0,False
7,0.729513,0.827724,1.745274e+09,6.994986e+14,685.287279,0,True
8,0.742133,0.174678,1.770775e+09,7.116715e+14,703.823694,0,False
9,0.016692,0.266514,1.763173e+09,7.078636e+14,696.695507,0,False


In [9]:
results1.describe()

,Window to Wall Ratio,Thickness,Electricity Usage,Electricity Variance,CO2:Facility,violation
count,10.000000,10.000000,1.000000e+01,1.000000e+01,10.000000,10.0
mean,0.482793,0.491079,1.756075e+09,7.047309e+14,692.251167,0.0
std,0.286756,0.251906,8.547875e+06,3.906972e+12,6.158278,0.0
min,0.016692,0.174678,1.745048e+09,6.994986e+14,685.014693,0.0
25%,0.274470,0.282334,1.751231e+09,7.023986e+14,688.355285,0.0
50%,0.510068,0.479517,1.754369e+09,7.038955e+14,690.508397,0.0
75%,0.738978,0.603193,1.762211e+09,7.073746e+14,696.325010,0.0
max,0.800154,0.924937,1.770775e+09,7.116715e+14,703.823694,0.0


In [10]:
# this cell runs the optimisation
results2 = NSGAII(evaluator, evaluations=10, population_size=10)
results2

,Window to Wall Ratio,Thickness,Electricity Usage,Electricity Variance,CO2:Facility,violation,pareto-optimal
0,0.390209,0.538738,1.753104e+09,7.033373e+14,689.441045,0,False
1,0.876055,0.398167,1.757236e+09,7.050660e+14,692.248612,0,False
2,0.339757,0.136475,1.774891e+09,7.141222e+14,707.556386,0,False
3,0.037267,0.177210,1.770534e+09,7.115455e+14,703.576742,0,False
4,0.113247,0.560691,1.751556e+09,7.022946e+14,689.489753,0,False
5,0.766694,0.783650,1.746680e+09,7.004682e+14,686.122813,0,False
6,0.587240,0.892875,1.743273e+09,6.987372e+14,684.359600,0,True
7,0.298284,0.707602,1.748879e+09,7.021818e+14,687.583670,0,False
8,0.235723,0.413703,1.756937e+09,7.048862e+14,691.947934,0,False
9,0.061911,0.046747,1.797665e+09,7.307813e+14,735.090515,0,False


In [11]:
results2.describe()

,Window to Wall Ratio,Thickness,Electricity Usage,Electricity Variance,CO2:Facility,violation
count,10.000000,10.000000,1.000000e+01,1.000000e+01,10.000000,10.0
mean,0.370639,0.465586,1.760076e+09,7.073420e+14,696.741707,0.0
std,0.289961,0.284965,1.655982e+07,9.508388e+12,15.395930,0.0
min,0.037267,0.046747,1.743273e+09,6.987372e+14,684.359600,0.0
25%,0.143866,0.232449,1.749548e+09,7.022100e+14,688.048013,0.0
50%,0.319021,0.476221,1.755021e+09,7.041118e+14,690.718843,0.0
75%,0.537982,0.670875,1.767210e+09,7.099256e+14,700.744709,0.0
max,0.876055,0.892875,1.797665e+09,7.307813e+14,735.090515,0.0


## Get available objectives

The user can use print_available_outputs to print out the available objectives for this building

In [12]:
from besos.eplus_funcs import print_available_outputs

building = ef.get_building(mode='idf')
print_available_outputs(building, version='9.0.1', name='facility', frequency='monthly')

['Electricity:Facility', 'Monthly']
['Gas:Facility', 'Monthly']
['CO2:Facility', 'Monthly']
['CO:Facility', 'Monthly']
['CH4:Facility', 'Monthly']
['NOx:Facility', 'Monthly']
['N2O:Facility', 'Monthly']
['SO2:Facility', 'Monthly']
['PM:Facility', 'Monthly']
['PM10:Facility', 'Monthly']
['PM2.5:Facility', 'Monthly']
['NH3:Facility', 'Monthly']
['NMVOC:Facility', 'Monthly']
['Hg:Facility', 'Monthly']
['Pb:Facility', 'Monthly']
['WaterEnvironmentalFactors:Facility', 'Monthly']
['Nuclear High:Facility', 'Monthly']
['Nuclear Low:Facility', 'Monthly']
['Carbon Equivalent:Facility', 'Monthly']
